## Web Scrapping using Selenium

##### Disclaimer: this notebook is created for academic purposes only, there is no affiliation with any institution.

In [3]:
pip install selenium


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

     |████████████████████████████████| 985 kB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 358 kB 104.5 MB/s eta 0:00:01
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
     |████████████████████████████████| 160 kB 104.7 MB/s eta 0:00:01
  Using cached h11-0.13.0-py3-none-any.whl (58 kB)
  Attempt

In [4]:
pip install webdriver-manager


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install fake_useragent


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13486 sha256=31b762f38ebe77ff070b31fcedff2bc9511f35432129f70329cab66f9e135910
  Stored in directory: /home/faculty/.cache/pip/wheels/a0/b8/b7/8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent
Note: you may 

In [6]:
# Import selenium packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from tqdm import tqdm

# Import general packages 
import numpy as np
import pandas as pd
import time

# Package for feature transformation
from sklearn.preprocessing import OneHotEncoder

In [9]:
# Import webdriver

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

### [NEW] Scrape leaderboard stats (serve leader, return leader, and under pressure leader) 

#### Serve Leaders Versus All Players On All Surfaces For Career

In [10]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_911/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [11]:
# Define web pages
leaderboard_serve_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=serve&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'
leaderboard_pressure_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [12]:
# Open browser serve leaderboard
browser.get(leaderboard_serve_leader)

In [13]:
# Define class for serve leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table')
players=all_data.find_elements(by=By.CLASS_NAME, value='player-leaderboard-info')

In [14]:
# Create dataframe
player_dict=dict()
serve_rating_idx_dict=dict()
rate_1st_serve_dict=dict()
rate_1st_serve_points_dict=dict()
rate_2nd_serve_points_dict=dict()
rate_service_game_won_dict=dict()
avg_aces_per_match_dict=dict()
avg_double_faults_per_match_dict=dict()

player_df = pd.DataFrame()
serve_rating_df = pd.DataFrame()
rate_1st_serve_df=pd.DataFrame()
rate_1st_serve_points_df=pd.DataFrame()
rate_2nd_serve_points_df=pd.DataFrame()
rate_service_game_won_df=pd.DataFrame()
avg_aces_per_match_df=pd.DataFrame()
avg_double_faults_per_match_df=pd.DataFrame()

In [15]:
# Fetch all data
i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    serve_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    serve_rating_idx_dict['serve_rating']=serve_rating.text
    serve_rating_df = serve_rating_df.append(serve_rating_idx_dict, ignore_index=True)

    rate_1st_serve=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_1st_serve_dict['rate_1st_serve']=rate_1st_serve.text
    rate_1st_serve_df = rate_1st_serve_df.append(rate_1st_serve_dict, ignore_index=True)

    rate_1st_serve_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_1st_serve_points_dict['rate_1st_serve_points']=rate_1st_serve_points.text
    rate_1st_serve_points_df = rate_1st_serve_points_df.append(rate_1st_serve_points_dict, ignore_index=True)

    rate_2nd_serve_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    rate_2nd_serve_points_dict['rate_2nd_serve_points']=rate_2nd_serve_points.text
    rate_2nd_serve_points_df = rate_2nd_serve_points_df.append(rate_2nd_serve_points_dict, ignore_index=True)

    rate_service_game_won=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    rate_service_game_won_dict['rate_service_game_won']=rate_service_game_won.text
    rate_service_game_won_df = rate_service_game_won_df.append(rate_service_game_won_dict, ignore_index=True)

    avg_aces_per_match=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[8]')
    avg_aces_per_match_dict['avg_aces_per_match']=avg_aces_per_match.text
    avg_aces_per_match_df = avg_aces_per_match_df.append(avg_aces_per_match_dict, ignore_index=True)

    avg_double_faults_per_match=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[9]')
    avg_double_faults_per_match_dict['avg_double_faults_per_match']=avg_double_faults_per_match.text
    avg_double_faults_per_match_df = avg_double_faults_per_match_df.append(avg_double_faults_per_match_dict, ignore_index=True)
    
    i=i+1

In [16]:
# Concat all columns
df_serve_leaderboard=pd.concat([player_df,serve_rating_df,rate_1st_serve_df,rate_1st_serve_points_df,rate_2nd_serve_points_df,
                                rate_service_game_won_df,avg_aces_per_match_df,avg_double_faults_per_match_df], axis=1)

In [17]:
# Removing unwanted character
df_serve_leaderboard['rate_1st_serve'] = df_serve_leaderboard['rate_1st_serve'].str.replace('%', '')
df_serve_leaderboard['rate_1st_serve_points'] = df_serve_leaderboard['rate_1st_serve_points'].str.replace('%', '')
df_serve_leaderboard['rate_2nd_serve_points'] = df_serve_leaderboard['rate_2nd_serve_points'].str.replace('%', '')
df_serve_leaderboard['rate_service_game_won'] = df_serve_leaderboard['rate_service_game_won'].str.replace('%', '')

In [18]:
# Change datatype
df_serve_leaderboard['serve_rating'] = df_serve_leaderboard['serve_rating'].astype(np.float64)
df_serve_leaderboard['rate_1st_serve'] = df_serve_leaderboard['rate_1st_serve'].astype(np.float64)
df_serve_leaderboard['rate_1st_serve_points'] = df_serve_leaderboard['rate_1st_serve_points'].astype(np.float64)
df_serve_leaderboard['rate_2nd_serve_points'] = df_serve_leaderboard['rate_2nd_serve_points'].astype(np.float64)
df_serve_leaderboard['rate_service_game_won'] = df_serve_leaderboard['rate_service_game_won'].astype(np.float64)
df_serve_leaderboard['avg_aces_per_match'] = df_serve_leaderboard['avg_aces_per_match'].astype(np.float64)
df_serve_leaderboard['avg_double_faults_per_match'] = df_serve_leaderboard['avg_double_faults_per_match'].astype(np.float64)

In [19]:
df_serve_leaderboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player                       748 non-null    object 
 1   serve_rating                 748 non-null    float64
 2   rate_1st_serve               748 non-null    float64
 3   rate_1st_serve_points        748 non-null    float64
 4   rate_2nd_serve_points        748 non-null    float64
 5   rate_service_game_won        748 non-null    float64
 6   avg_aces_per_match           748 non-null    float64
 7   avg_double_faults_per_match  748 non-null    float64
dtypes: float64(7), object(1)
memory usage: 46.9+ KB


In [20]:
df_serve_leaderboard

,player,serve_rating,rate_1st_serve,rate_1st_serve_points,rate_2nd_serve_points,rate_service_game_won,avg_aces_per_match,avg_double_faults_per_match
0,John Isner,312.1,69.2,78.7,56.1,91.9,18.5,2.3
1,Ivo Karlovic,309.8,65.5,82.7,53.3,92.0,19.8,3.5
2,Reilly Opelka,304.5,64.2,79.3,55.1,90.4,18.4,2.9
3,Milos Raonic,302.8,62.9,81.4,55.1,91.2,15.5,3.3
4,Andy Roddick,299.9,64.8,79.3,56.0,90.1,11.7,2.0
...,...,...,...,...,...,...,...,...
743,Razvan Sabau,226.2,59.3,62.0,44.5,62.1,2.4,4.1
744,Marian Vajda,223.4,54.3,61.5,47.6,61.7,2.3,4.0
745,Daniel Elsner,222.4,54.1,63.5,44.9,61.8,2.7,4.6
746,Bart Wuyts,221.6,70.1,57.7,40.9,56.7,0.4,4.2


#### Return Leaders Versus All Players On All Surfaces For Career

In [21]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_911/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [22]:
# Define web pages
leaderboard_return_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=return&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [23]:
# Open browser return leaderboard
browser.get(leaderboard_return_leader)

In [24]:
# Define class for return leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table/tbody')
players=all_data.find_elements(by=By.CLASS_NAME, value='player-leaderboard-info')

In [35]:
# Create dataframe
player_dict=dict()
return_rating_dict=dict()
rate_1st_return_points_dict=dict()
rate_2nd_return_points_dict=dict()
return_game_won_dict=dict()
break_point_dict=dict()

player_df = pd.DataFrame()
return_rating_df = pd.DataFrame()
rate_1st_return_points_df=pd.DataFrame()
rate_2nd_return_points_df=pd.DataFrame()
return_game_won_df=pd.DataFrame()
break_point_df=pd.DataFrame()

In [36]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    return_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    return_rating_dict['return_rating']=return_rating.text
    return_rating_df = return_rating_df.append(return_rating_dict, ignore_index=True)

    rate_1st_return_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_1st_return_points_dict['rate_1st_return_points']=rate_1st_return_points.text
    rate_1st_return_points_df = rate_1st_return_points_df.append(rate_1st_return_points_dict, ignore_index=True)

    rate_2nd_return_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_2nd_return_points_dict['rate_2nd_return_points']=rate_2nd_return_points.text
    rate_2nd_return_points_df = rate_2nd_return_points_df.append(rate_2nd_return_points_dict, ignore_index=True)

    return_game_won=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    return_game_won_dict['return_game_won']=return_game_won.text
    return_game_won_df = return_game_won_df.append(return_game_won_dict, ignore_index=True)

    break_point=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    break_point_dict['break_point']=break_point.text
    break_point_df = break_point_df.append(break_point_dict, ignore_index=True)

    i=i+1

In [37]:
# Concat all columns
df_return_leaderboard=pd.concat([player_df,return_rating_df,rate_1st_return_points_df,rate_2nd_return_points_df,
                                return_game_won_df,break_point_df], axis=1)

In [38]:
# Removing unwanted character
df_return_leaderboard['return_rating'] = df_return_leaderboard['return_rating'].str.replace('%', '')
df_return_leaderboard['rate_1st_return_points'] = df_return_leaderboard['rate_1st_return_points'].str.replace('%', '')
df_return_leaderboard['rate_2nd_return_points'] = df_return_leaderboard['rate_2nd_return_points'].str.replace('%', '')
df_return_leaderboard['return_game_won'] = df_return_leaderboard['return_game_won'].str.replace('%', '')
df_return_leaderboard['break_point'] = df_return_leaderboard['break_point'].str.replace('%', '')

In [39]:
# Change datatype
df_return_leaderboard['return_rating'] = df_return_leaderboard['return_rating'].astype(np.float64)
df_return_leaderboard['rate_1st_return_points'] = df_return_leaderboard['rate_1st_return_points'].astype(np.float64)
df_return_leaderboard['rate_2nd_return_points'] = df_return_leaderboard['rate_2nd_return_points'].astype(np.float64)
df_return_leaderboard['rate_2nd_return_points'] = df_return_leaderboard['rate_2nd_return_points'].astype(np.float64)
df_return_leaderboard['return_game_won'] = df_return_leaderboard['return_game_won'].astype(np.float64)
df_return_leaderboard['break_point'] = df_return_leaderboard['break_point'].astype(np.float64)

In [40]:
df_return_leaderboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player                  748 non-null    object 
 1   return_rating           748 non-null    float64
 2   rate_1st_return_points  748 non-null    float64
 3   rate_2nd_return_points  748 non-null    float64
 4   return_game_won         748 non-null    float64
 5   break_point             748 non-null    float64
dtypes: float64(5), object(1)
memory usage: 35.2+ KB


In [41]:
df_return_leaderboard

,player,return_rating,rate_1st_return_points,rate_2nd_return_points,return_game_won,break_point
0,Guillermo Coria,171.5,36.1,54.4,35.3,45.7
1,Rafael Nadal,168.2,34.2,55.4,33.6,45.0
2,Guillermo Perez-Roldan,168.1,35.8,53.2,32.5,46.6
3,Alberto Berasategui,166.6,33.6,56.3,32.4,44.3
4,Franco Davin,166.3,34.8,53.2,32.8,45.5
...,...,...,...,...,...,...
743,Reilly Opelka,104.9,20.7,42.3,10.2,31.7
744,John Isner,104.6,22.4,41.9,10.2,30.1
745,Wayne Arthurs,103.9,23.4,40.5,10.0,30.0
746,Ivo Karlovic,100.1,21.2,40.2,8.5,30.2


#### Under Pressure Leaders Versus All Players On All Surfaces For Career

https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false

In [42]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_911/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [43]:
# Define web pages
leaderboard_pressure_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [44]:
# Open browser under pressure leaderboard
browser.get(leaderboard_pressure_leader)

In [45]:
# Define class for under pressure leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table/tbody')
players=all_data.find_elements(by=By.CLASS_NAME, value='stats-player-name')

In [46]:
# Create dataframe
player_dict=dict()
pressure_rating_dict=dict()
rate_break_points_converted_dict=dict()
rate_break_points_saved_dict=dict()
rate_tie_break_dict=dict()
rate_deciding_set_dict=dict()

player_df = pd.DataFrame()
pressure_rating_df = pd.DataFrame()
rate_break_points_converted_df=pd.DataFrame()
rate_break_points_saved_df=pd.DataFrame()
rate_tie_break_df=pd.DataFrame()
rate_deciding_set_df=pd.DataFrame()

In [47]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    pressure_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    pressure_rating_dict['pressure_rating']=pressure_rating.text
    pressure_rating_df = pressure_rating_df.append(pressure_rating_dict, ignore_index=True)

    rate_break_points_converted=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_break_points_converted_dict['rate_break_points_converted']=rate_break_points_converted.text
    rate_break_points_converted_df = rate_break_points_converted_df.append(rate_break_points_converted_dict, ignore_index=True)

    rate_break_points_saved=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_break_points_saved_dict['rate_break_points_saved']=rate_break_points_saved.text
    rate_break_points_saved_df = rate_break_points_saved_df.append(rate_break_points_saved_dict, ignore_index=True)

    rate_tie_break=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    rate_tie_break_dict['rate_tie_break']=rate_tie_break.text
    rate_tie_break_df = rate_tie_break_df.append(rate_tie_break_dict, ignore_index=True)

    rate_deciding_set=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    rate_deciding_set_dict['rate_deciding_set']=rate_deciding_set.text
    rate_deciding_set_df = rate_deciding_set_df.append(rate_deciding_set_dict, ignore_index=True)

    i=i+1

In [48]:
# Concat all columns
df_under_pressure_leaderboard=pd.concat([player_df,pressure_rating_df,rate_break_points_converted_df,rate_break_points_saved_df,rate_tie_break_df,
                                rate_deciding_set_df], axis=1)

In [49]:
# Removing unwanted character
df_under_pressure_leaderboard['rate_break_points_converted'] = df_under_pressure_leaderboard['rate_break_points_converted'].str.replace('%', '')
df_under_pressure_leaderboard['rate_break_points_saved'] = df_under_pressure_leaderboard['rate_break_points_saved'].str.replace('%', '')
df_under_pressure_leaderboard['rate_tie_break'] = df_under_pressure_leaderboard['rate_tie_break'].str.replace('%', '')
df_under_pressure_leaderboard['rate_deciding_set'] = df_under_pressure_leaderboard['rate_deciding_set'].str.replace('%', '')

In [50]:
# Change datatype
df_under_pressure_leaderboard['pressure_rating'] = df_under_pressure_leaderboard['pressure_rating'].astype(np.float64)
df_under_pressure_leaderboard['rate_break_points_converted'] = df_under_pressure_leaderboard['rate_break_points_converted'].astype(np.float64)
df_under_pressure_leaderboard['rate_break_points_saved'] = df_under_pressure_leaderboard['rate_break_points_saved'].astype(np.float64)
df_under_pressure_leaderboard['rate_tie_break'] = df_under_pressure_leaderboard['rate_tie_break'].astype(np.float64)
df_under_pressure_leaderboard['rate_deciding_set'] = df_under_pressure_leaderboard['rate_deciding_set'].astype(np.float64)

In [51]:
df_under_pressure_leaderboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player                       748 non-null    object 
 1   pressure_rating              748 non-null    float64
 2   rate_break_points_converted  748 non-null    float64
 3   rate_break_points_saved      748 non-null    float64
 4   rate_tie_break               748 non-null    float64
 5   rate_deciding_set            748 non-null    float64
dtypes: float64(5), object(1)
memory usage: 35.2+ KB


In [52]:
df_under_pressure_leaderboard

,player,pressure_rating,rate_break_points_converted,rate_break_points_saved,rate_tie_break,rate_deciding_set
0,Novak Djokovic,247.5,44.3,65.5,64.4,73.3
1,Federico Coria,242.9,42.8,57.5,70.6,72.0
2,Rafael Nadal,242.0,45.0,66.6,60.9,69.5
3,Pete Sampras,241.9,40.6,67.9,64.1,69.3
4,Roger Federer,238.9,41.1,67.3,65.3,65.2
...,...,...,...,...,...,...
743,Patrik Fredriksson,137.4,37.1,50.3,50.0,0.0
744,Julian Knowle,136.9,35.9,57.0,16.7,27.3
745,Andrew Sznajder,136.6,31.6,56.4,37.5,11.1
746,Yuki Bhambri,134.3,42.3,52.9,9.1,30.0


### [NEW] Scrape Ranks Single

In [53]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_911/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [54]:
# Define web pages
ranks = 'https://www.atptour.com/en/rankings/singles?rankRange=1-5000&rankDate=2022-04-25'

In [55]:
# Open browser rankings page
browser.get(ranks)

In [56]:
# Define class for single ranks
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[2]/div/table/tbody')
players=all_data.find_elements(by=By.CLASS_NAME, value='player-cell-wrapper')

In [57]:
# Create dataframe
player_dict=dict()
age_dict=dict()
points_dict=dict()
tourn_played_dict=dict()
next_best_dict=dict()

player_df = pd.DataFrame()
age_df = pd.DataFrame()
points_df = pd.DataFrame()
tourn_played_df = pd.DataFrame()
next_best_df = pd.DataFrame()

In [58]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    age=browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[5]')
    age_dict['age']=age.text
    age_df = age_df.append(age_dict, ignore_index=True)
    
    points=browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[6]/a')
    points_dict['points']=points.text
    points_df = points_df.append(points_dict, ignore_index=True)
    
    tourn_played=browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[8]/a')
    tourn_played_dict['tourn_played']=tourn_played.text
    tourn_played_df = tourn_played_df.append(tourn_played_dict, ignore_index=True)

    next_best=browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[10]')
    next_best_dict['next_best']=next_best.text
    next_best_df = next_best_df.append(next_best_dict, ignore_index=True)


    i=i+1

In [59]:
# Concat all columns
df_rankings=pd.concat([player_df,age_df,points_df,tourn_played_df,next_best_df], axis=1)

In [60]:
# Removing unwanted character
df_rankings['points'] = df_rankings['points'].str.replace(',', '')


In [61]:
# Change data type
df_rankings['age'] = pd.to_numeric(df_rankings['age'])
df_rankings['points'] = pd.to_numeric(df_rankings['points'])
df_rankings['tourn_played'] = pd.to_numeric(df_rankings['tourn_played'])
df_rankings['next_best'] = pd.to_numeric(df_rankings['next_best'])

In [62]:
df_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2174 entries, 0 to 2173
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player        2174 non-null   object 
 1   age           2173 non-null   float64
 2   points        2174 non-null   int64  
 3   tourn_played  2174 non-null   int64  
 4   next_best     2174 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 85.0+ KB


In [63]:
df_rankings

,player,age,points,tourn_played,next_best
0,Novak Djokovic,34.0,8400,13,0
1,Daniil Medvedev,26.0,8080,21,0
2,Alexander Zverev,25.0,7465,22,0
3,Rafael Nadal,35.0,6435,10,0
4,Stefanos Tsitsipas,23.0,5770,26,45
...,...,...,...,...,...
2169,Dominik Kellovsky,25.0,1,10,0
2170,Ren Nakamura,27.0,1,10,0
2171,Valentin Vanta,24.0,1,10,0
2172,Umut Akkoyun,21.0,1,13,0


### [NEW] Merge tournament files and create new variable for win/lose

In [246]:
import os
import pandas as pd
cwd = os.path.abspath('') 
files = os.listdir(cwd) 

In [247]:
cwd

'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Individual Coursework/webscrape'

In [248]:
files

['2021.xlsx',
 'atp_web_scrap_final_extend_v2.ipynb',
 '2017.xlsx',
 '2016.xlsx',
 '.DS_Store',
 '2020.xlsx',
 'atp_web_scrap_final_extend_v3.ipynb',
 '2013.xlsx',
 '2019.xlsx',
 '.ipynb_checkpoints',
 'csv_result',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

In [249]:
df_atp_match = pd.DataFrame()

for file in files:                         # loop through Excel files
    if file.endswith('.xlsx'):
        excel_file = pd.ExcelFile(file)
        sheets = excel_file.sheet_names
        for sheet in sheets:               # loop through sheets inside an Excel file
            df = excel_file.parse(sheet_name = sheet)
            df_atp_match = df_atp_match.append(df)

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [250]:
df_atp_match.columns=df_atp_match.columns.str.strip().str.lower()

In [251]:
# Identify missing value
percent_missing = df_atp_match.isnull().sum() * 100 / len(df_atp_match)
missing_value_data = pd.DataFrame({'percent_missing': percent_missing})
missing_value_data = missing_value_data.sort_values('percent_missing', ascending = False)
missing_value_data['percent_missing'] = pd.Series(["{0:.2f}%".format(val) for val in missing_value_data['percent_missing']], index = missing_value_data.index)
missing_value_data

,percent_missing
l5,96.38%
w5,96.38%
w4,90.31%
l4,90.31%
sjw,77.94%
sjl,77.94%
w3,51.84%
l3,51.84%
lbw,35.34%
lbl,35.32%


In [252]:
df_atp_match.shape

(23098, 42)

In [253]:
# Drop variables with a relatively high missing value %
df_atp_match = df_atp_match.drop(['l5','w5','w4','l4','sjw','sjl','w3','l3','lbw','lbl','exw','exl'],axis = 1)

In [254]:
# Drop rows which variable has a very low missing value %
df_atp_match = df_atp_match.dropna(how = 'any', subset = ['wpts','wrank','maxl','avgw','avgl','lrank','lpts','b365l',
                                                          'b365w','psl','psw','l1','w1','wsets','lsets','l2','w2'])

In [255]:
df_atp_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22553 entries, 0 to 2636
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   atp         22553 non-null  int64         
 1   location    22553 non-null  object        
 2   tournament  22553 non-null  object        
 3   date        22553 non-null  datetime64[ns]
 4   series      22553 non-null  object        
 5   court       22553 non-null  object        
 6   surface     22553 non-null  object        
 7   round       22553 non-null  object        
 8   best of     22553 non-null  int64         
 9   winner      22553 non-null  object        
 10  loser       22553 non-null  object        
 11  wrank       22553 non-null  float64       
 12  lrank       22553 non-null  float64       
 13  wpts        22553 non-null  float64       
 14  lpts        22553 non-null  float64       
 15  w1          22553 non-null  float64       
 16  l1          22553 non-n

In [256]:
df_atp_match.head().T

,0,1,2,3,4
atp,1,1,1,1,1
location,Antalya,Antalya,Antalya,Antalya,Antalya
tournament,Antalya Open,Antalya Open,Antalya Open,Antalya Open,Antalya Open
date,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00
series,ATP250,ATP250,ATP250,ATP250,ATP250
court,Indoor,Indoor,Indoor,Indoor,Indoor
surface,Hard,Hard,Hard,Hard,Hard
round,1st Round,1st Round,1st Round,1st Round,1st Round
best of,3,3,3,3,3
winner,Basilashvili N.,Celikbilek A.,Ruusuvuori E.,Bublik A.,Goffin D.


In [257]:
# Saving dataset into csv file
df_atp_match.to_csv(r'/folder/atp_match.csv',
                  index=False, decimal=".")

In [76]:
# Split winner and loser name
df_atp_match[['winner_last','winner_first']]=df_atp_match['winner'].str.split(' ',n=1, expand=True)
df_atp_match[['loser_last','loser_first']]=df_atp_match['loser'].str.split(' ',n=1, expand=True)

In [77]:
df_atp_match.head().T

,0,1,2,3,4
atp,1,1,1,1,1
location,Antalya,Antalya,Antalya,Antalya,Antalya
tournament,Antalya Open,Antalya Open,Antalya Open,Antalya Open,Antalya Open
date,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00,2021-01-07 00:00:00
series,ATP250,ATP250,ATP250,ATP250,ATP250
court,Indoor,Indoor,Indoor,Indoor,Indoor
surface,Hard,Hard,Hard,Hard,Hard
round,1st Round,1st Round,1st Round,1st Round,1st Round
best of,3,3,3,3,3
winner,Basilashvili N.,Celikbilek A.,Ruusuvuori E.,Bublik A.,Goffin D.


#### Merge all dataset
1. merge player's feature dataset
- rankings
- serve leaderboard
- return leaderboard
- under pressure leaderboard

2. check missing value 
3. create dependent variable


In [98]:
# Merge all player's feature dataset
temp_merge_1=pd.merge(df_serve_leaderboard,df_rankings,on='player',how='inner')
temp_merge_2=pd.merge(temp_merge_1,df_return_leaderboard,on='player',how='inner')
temp_merge_3=pd.merge(temp_merge_2,df_under_pressure_leaderboard,on='player',how='inner')

df_player_feature=temp_merge_3.copy()

In [101]:
# Identify missing value
percent_missing = df_player_feature.isnull().sum() * 100 / len(df_player_feature)
missing_value_data = pd.DataFrame({'percent_missing': percent_missing})
missing_value_data = missing_value_data.sort_values('percent_missing', ascending = False)
missing_value_data['percent_missing'] = pd.Series(["{0:.2f}%".format(val) for val in missing_value_data['percent_missing']], index = missing_value_data.index)
missing_value_data

,percent_missing
player,0.00%
serve_rating,0.00%
rate_tie_break,0.00%
rate_break_points_saved,0.00%
rate_break_points_converted,0.00%
pressure_rating,0.00%
break_point,0.00%
return_game_won,0.00%
rate_2nd_return_points,0.00%
rate_1st_return_points,0.00%


In [102]:
# Make copy from original dataset for data preparation
df_fp = df_player_feature.copy()
df_match = df_atp_match.copy()

In [106]:
# Get player's last name to be the identifier when combining match data
df_fp['player_last_name']=df_fp['player'].apply(lambda x: x.split(" ")[-1])

#### Merge match dataset with player's feature dataset

In [227]:
# Merge match dataset to player's feature dataset
df_merge_winner=pd.merge(df_atp_match, df_fp, how='left', left_on=['winner_last'], right_on=['player_last_name'])
df_merge_loser=pd.merge(df_merge_winner, df_fp, how='left', left_on=['loser_last'], right_on=['player_last_name'])

df_main=df_merge_loser.copy()

In [228]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23732 entries, 0 to 23731
Data columns (total 80 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   atp                            23732 non-null  int64         
 1   location                       23732 non-null  object        
 2   tournament                     23732 non-null  object        
 3   date                           23732 non-null  datetime64[ns]
 4   series                         23732 non-null  object        
 5   court                          23732 non-null  object        
 6   surface                        23732 non-null  object        
 7   round                          23732 non-null  object        
 8   best of                        23732 non-null  int64         
 9   winner                         23732 non-null  object        
 10  loser                          23732 non-null  object        
 11  wrank          

#### Remove unnecessary columns for model building

In [229]:
# Identify missing value
percent_missing = df_main.isnull().sum() * 100 / len(df_main)
missing_value_data = pd.DataFrame({'percent_missing': percent_missing})
missing_value_data = missing_value_data.sort_values('percent_missing', ascending = False)
missing_value_data['percent_missing'] = pd.Series(["{0:.2f}%".format(val) for val in missing_value_data['percent_missing']], index = missing_value_data.index)
missing_value_data

,percent_missing
player_last_name_y,22.16%
next_best_y,22.16%
player_y,22.16%
serve_rating_y,22.16%
rate_1st_serve_y,22.16%
...,...
winner_last,0.00%
winner_first,0.00%
loser_last,0.00%
loser_first,0.00%


In [230]:
# Drop rows which variable has low missing value %
removed_rows=['rate_deciding_set_y','tourn_played_y','player_y','serve_rating_y','rate_1st_serve_y',
'rate_1st_serve_points_y','rate_2nd_serve_points_y','avg_aces_per_match_y',
'avg_double_faults_per_match_y','age_y','points_y','rate_service_game_won_y',
'next_best_y','rate_1st_return_points_y','rate_2nd_return_points_y','return_game_won_y',
'break_point_y','pressure_rating_y','rate_break_points_converted_y','rate_break_points_saved_y',
'rate_tie_break_y','return_rating_y','break_point_x','next_best_x','return_rating_x',
'rate_1st_return_points_x','rate_2nd_return_points_x','return_game_won_x',
'rate_deciding_set_x','pressure_rating_x','rate_break_points_converted_x',
'rate_break_points_saved_x','rate_tie_break_x','points_x','tourn_played_x','age_x',
'avg_aces_per_match_x','rate_service_game_won_x','rate_2nd_serve_points_x','rate_1st_serve_points_x',
'rate_1st_serve_x','serve_rating_x','player_x','avg_double_faults_per_match_x']

df_main = df_main.dropna(how = 'any', subset = removed_rows)

In [231]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15466 entries, 2 to 23731
Data columns (total 80 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   atp                            15466 non-null  int64         
 1   location                       15466 non-null  object        
 2   tournament                     15466 non-null  object        
 3   date                           15466 non-null  datetime64[ns]
 4   series                         15466 non-null  object        
 5   court                          15466 non-null  object        
 6   surface                        15466 non-null  object        
 7   round                          15466 non-null  object        
 8   best of                        15466 non-null  int64         
 9   winner                         15466 non-null  object        
 10  loser                          15466 non-null  object        
 11  wrank          

Notes: x refers to winner, y refers to loser. In the next section, the name will be changed.

### Feature Transformation

There are three processes in the following sections: 
1. Features Selection
- Some variables will be removed because they are related to the match scores, which are unknown before a match start. It was kept for data exploration only. 
- Some variables that do not affect the chance of a player winning a game will be removed, e.g. winner_ioc, loser_ioc, tourney_id, tourney_name.
- Some variables that have already been described by other variables will be removed, e.g. winner_id and loser_id, defined by winner_name and loser_name. 

2. Create Dependent Variable
- Current dataset does not have a variable as the target feature.
- A new feature is created to capture a binary value with '0' if Player 1 wins and '1' if Player 2 wins. This is the feature the model will predict. 
- Below are the steps to create the feature:
    - 'win_result' will be labelled:
        - '0' if Player 1 wins
        - '1' if Player 2 wins
    - To avoid the biases of having Player 1 always be the winner, Player 1 and Player 2 will be called alphabetically, and win_result will be assigned accordingly.
    - The data will be subset according to the win_result value
    - The variables then renamed to Player 1 & Player 2, where the rule inversed between win_result 0 and 1. 
    - The two subsets will be concatenated to form one complete dataset again.
    - Class balance will be checked later to ensure roughly half of the matches, Player 1 is the winner, and the same goes with Player 2 to avoid bias.

3. Feature Transformation
- In this section, the categorical features will be encoded. 

In [232]:
df_model=df_main.copy()

#### Features Selection

In [233]:
# Drop unnecessary/repeating columns
df_model = df_model.drop(['atp','location','tournament','comment','winner_last','winner_first','loser_last','loser_first',
                       'player_last_name_x','player_last_name_y','w1','w2','l1','l2','wsets','lsets',
                         ],axis = 1)

#### Create dependent variable

In [234]:
# Assign value to win_result based on the alphabetical order
df_model["win_result"] = df_model.apply(lambda row: 1 if row["player_x"] > row["player_y"] else 0, axis=1)
df_model[["winner","player_x", "loser","player_y", "win_result"]].head(10)

,winner,player_x,loser,player_y,win_result
2,Ruusuvuori E.,Emil Ruusuvuori,Vesely J.,Jiri Vesely,0
3,Bublik A.,Alexander Bublik,Caruso S.,Salvatore Caruso,0
4,Goffin D.,David Goffin,Herbert P.H.,Pierre-Hugues Herbert,0
5,Travaglia S.,Stefano Travaglia,Kecmanovic M.,Miomir Kecmanovic,1
11,Chardy J.,Jeremy Chardy,Albot R.,Radu Albot,0
18,Travaglia S.,Stefano Travaglia,Ruusuvuori E.,Emil Ruusuvuori,1
20,Chardy J.,Jeremy Chardy,Fognini F.,Fabio Fognini,1
23,Goffin D.,David Goffin,Travaglia S.,Stefano Travaglia,0
24,Chardy J.,Jeremy Chardy,Struff J.L.,Jan-Lennard Struff,1
25,Bublik A.,Alexander Bublik,Berrettini M.,Matteo Berrettini,0


In [235]:
# Class Balance

# Calculate current class balance
print ('Number of Player 1 Win is: {}'.format((df_model.win_result == 0).sum()))
print ('Number of Player 1 Win is: {:.2f} %'.format((df_model.win_result == 0).sum()/len(df_model)*100))
print ('Number of Player 2 Win is: {}'.format((df_model.win_result == 1).sum()))
print ('Number of Player 2 Win is: {:.2f} %'.format((df_model.win_result == 1).sum()/len(df_model)*100))

Number of Player 1 Win is: 7729
Number of Player 1 Win is: 49.97 %
Number of Player 2 Win is: 7737
Number of Player 2 Win is: 50.03 %


In [236]:
# Create subset for each win_result
df_subset_a=df_model.loc[df_model['win_result']==0]
df_subset_b=df_model.loc[df_model['win_result']==1]

In [237]:
# Rename columns for each subsets based on win_result input

# Rename subset for win_result = 0
df_subset_a=df_subset_a.rename(columns={
    'player_x':'p1_name','serve_rating_x':'serve_rating_p1','rate_1st_serve_x':'rate_1st_serve_p1',
    'rate_1st_serve_points_x':'rate_1st_serve_points_p1','rate_2nd_serve_points_x':'rate_2nd_serve_points_p1',
    'rate_service_game_won_x':'rate_service_game_won_p1','avg_aces_per_match_x':'avg_aces_per_match_p1',
    'avg_double_faults_per_match_x':'avg_double_faults_per_match_p1','age_x':'age_p1',
    'points_x':'points_p1','tourn_played_x':'tourn_played_p1','next_best_x':'next_best_p1',
    'return_rating_x':'return_rating_p1','rate_1st_return_points_x':'rate_1st_return_points_p1',
    'rate_1st_return_points_x':'rate_1st_return_points_p1','rate_2nd_return_points_x':'rate_2nd_return_points_p1',
    'rate_2nd_return_points_x':'rate_2nd_return_points_p1','return_game_won_x':'return_game_won_p1',
    'break_point_x':'break_point_p1','pressure_rating_x':'pressure_rating_p1',
    'rate_break_points_converted_x':'rate_break_points_converted_p1',
    'rate_break_points_saved_x':'rate_break_points_saved_p1','rate_tie_break_x':'rate_tie_break_p1',
    'rate_deciding_set_x':'rate_deciding_set_p1','player_y':'p2_name',
    'serve_rating_y':'serve_rating_p2','rate_1st_serve_y':'rate_1st_serve_p2',
    'rate_1st_serve_points_y':'rate_1st_serve_points_p2','rate_2nd_serve_points_y':'rate_2nd_serve_points_p2',
    'rate_service_game_won_y':'rate_service_game_won_p2','avg_aces_per_match_y':'avg_aces_per_match_p2',
    'avg_aces_per_match_y':'avg_aces_per_match_p2','avg_double_faults_per_match_y':'avg_double_faults_per_match_p2',
    'age_y':'age_p2','points_y':'points_p2','tourn_played_y':'tourn_played_p2','next_best_y':'next_best_p2',
    'return_rating_y':'return_rating_p2','rate_1st_return_points_y':'rate_1st_return_points_p2',
    'rate_2nd_return_points_y':'rate_2nd_return_points_p2','return_game_won_y':'return_game_won_p2',
    'break_point_y':'break_point_p2','pressure_rating_y':'pressure_rating_p2','pressure_rating_y':'pressure_rating_p2',
    'rate_break_points_converted_y':'rate_break_points_converted_p2','rate_break_points_saved_y':'rate_break_points_saved_p2',
    'rate_tie_break_y':'rate_tie_break_p2','rate_deciding_set_y':'rate_deciding_set_p2',
    'wrank':'rank_p1','lrank':'rank_p2','b365w':'b365_p1','b365l':'b365_p2',
    'psw':'ps_p1','psl':'ps_p2','maxw':'max_p1','maxl':'max_p2','avgw':'avg_p1','avgl':'avg_p2'})

# Rename subset for win_result = 1 (inverse of df_subset_a)
df_subset_b=df_subset_b.rename(columns={
    'player_x':'p2_name','serve_rating_x':'serve_rating_p2','rate_1st_serve_x':'rate_1st_serve_p2',
    'rate_1st_serve_points_x':'rate_1st_serve_points_p2','rate_2nd_serve_points_x':'rate_2nd_serve_points_p2',
    'rate_service_game_won_x':'rate_service_game_won_p2','avg_aces_per_match_x':'avg_aces_per_match_p2',
    'avg_double_faults_per_match_x':'avg_double_faults_per_match_p2','age_x':'age_p2',
    'points_x':'points_p2','tourn_played_x':'tourn_played_p2','next_best_x':'next_best_p2',
    'return_rating_x':'return_rating_p2','rate_1st_return_points_x':'rate_1st_return_points_p2',
    'rate_1st_return_points_x':'rate_1st_return_points_p2','rate_2nd_return_points_x':'rate_2nd_return_points_p2',
    'rate_2nd_return_points_x':'rate_2nd_return_points_p2','return_game_won_x':'return_game_won_p2',
    'break_point_x':'break_point_p2','pressure_rating_x':'pressure_rating_p2',
    'rate_break_points_converted_x':'rate_break_points_converted_p2',
    'rate_break_points_saved_x':'rate_break_points_saved_p2','rate_tie_break_x':'rate_tie_break_p2',
    'rate_deciding_set_x':'rate_deciding_set_p2','player_y':'p1_name',
    'serve_rating_y':'serve_rating_p1','rate_1st_serve_y':'rate_1st_serve_p1',
    'rate_1st_serve_points_y':'rate_1st_serve_points_p1','rate_2nd_serve_points_y':'rate_2nd_serve_points_p1',
    'rate_service_game_won_y':'rate_service_game_won_p1','avg_aces_per_match_y':'avg_aces_per_match_p1',
    'avg_aces_per_match_y':'avg_aces_per_match_p1','avg_double_faults_per_match_y':'avg_double_faults_per_match_p1',
    'age_y':'age_p1','points_y':'points_p1','tourn_played_y':'tourn_played_p1','next_best_y':'next_best_p1',
    'return_rating_y':'return_rating_p1','rate_1st_return_points_y':'rate_1st_return_points_p1',
    'rate_2nd_return_points_y':'rate_2nd_return_points_p1','return_game_won_y':'return_game_won_p1',
    'break_point_y':'break_point_p1','pressure_rating_y':'pressure_rating_p1','pressure_rating_y':'pressure_rating_p1',
    'rate_break_points_converted_y':'rate_break_points_converted_p1','rate_break_points_saved_y':'rate_break_points_saved_p1',
    'rate_tie_break_y':'rate_tie_break_p1','rate_deciding_set_y':'rate_deciding_set_p1',
    'wrank':'rank_p2','lrank':'rank_p1','b365w':'b365_p2','b365l':'b365_p1',
    'psw':'ps_p2','psl':'ps_p1','maxw':'max_p2','maxl':'max_p1','avgw':'avg_p2','avgl':'avg_p1'})

# Concatenate subsets
df_model=pd.concat([df_subset_a,df_subset_b])

# Reorder based on tourney_date, so it is close enough to the original ordering
df_model = df_model.sort_values(by='date', ascending=True)

# Reset indexing
df_model=df_model.reset_index(drop=True)

In [238]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15466 entries, 0 to 15465
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            15466 non-null  datetime64[ns]
 1   series                          15466 non-null  object        
 2   court                           15466 non-null  object        
 3   surface                         15466 non-null  object        
 4   round                           15466 non-null  object        
 5   best of                         15466 non-null  int64         
 6   winner                          15466 non-null  object        
 7   loser                           15466 non-null  object        
 8   rank_p1                         15466 non-null  float64       
 9   rank_p2                         15466 non-null  float64       
 10  wpts                            15466 non-null  float64       
 11  lp

#### Feature transformation
In order to build the prediction model, the categorical feature needs to be encoded using OneHotEncoder.

In [240]:
# Create function to encode categorical features using OneHotEncoder
def cat_encoding(cat_cols):
    ohe=OneHotEncoder()
    cat_encoded=ohe.fit_transform(cat_cols)
    columns=ohe.get_feature_names(list(cat_cols))
    cat_cols=pd.DataFrame(cat_encoded.todense(),columns=columns)    
    return cat_cols

In [239]:
# Define list of columns with categorical value
cat_cols=['series','court','surface','round','best of']

# Separate columns that will not be tranformed
df_not_cat=df_model.drop(columns=cat_cols)
df_cat=df_model[cat_cols]

# Encode features
df_cat_encoded=cat_encoding(df_cat)

# Concatenate original and transformed dataframe
df_final=pd.concat([df_not_cat,df_cat_encoded],axis=1)

In [241]:
df_final.head().T

,0,1,2,3,4
date,2012-10-01 00:00:00,2012-10-01 00:00:00,2012-10-01 00:00:00,2012-10-02 00:00:00,2012-10-02 00:00:00
winner,Fognini F.,Gasquet R.,Tsonga J.W.,Wawrinka S.,Djokovic N.
loser,Robredo T.,Mayer F.,Monfils G.,Seppi A.,Rosol L.
rank_p1,19.0,41.0,42.0,22.0,46.0
rank_p2,18.0,10.0,8.0,9.0,1.0
...,...,...,...,...,...
round_Round Robin,0.0,0.0,0.0,0.0,0.0
round_Semifinals,0.0,0.0,0.0,0.0,0.0
round_The Final,0.0,0.0,0.0,0.0,0.0
best of_3,1.0,1.0,1.0,1.0,1.0


In [242]:
# shift column 'win_result' as the dependent variable to be the first column - sagemaker identifies dependent variable in the first column
first_column = df_final.pop('win_result')
df_final.insert(0, 'win_result', first_column)

# dataframe after shifting the column
print("After Shifting column to first position")
display(df_final)

After Shifting column to first position


,win_result,date,winner,loser,rank_p1,rank_p2,wpts,lpts,b365_p1,b365_p2,...,round_1st Round,round_2nd Round,round_3rd Round,round_4th Round,round_Quarterfinals,round_Round Robin,round_Semifinals,round_The Final,best of_3,best of_5
0,0,2012-10-01,Fognini F.,Robredo T.,19.0,18.0,1840.0,1855.0,3.00,1.36,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,2012-10-01,Gasquet R.,Mayer F.,41.0,10.0,3005.0,1030.0,3.75,1.25,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,2012-10-01,Tsonga J.W.,Monfils G.,42.0,8.0,3325.0,1030.0,2.75,1.40,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,2012-10-02,Wawrinka S.,Seppi A.,22.0,9.0,3150.0,1555.0,4.50,1.18,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,2012-10-02,Djokovic N.,Rosol L.,46.0,1.0,11120.0,951.0,21.00,1.01,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15461,0,2022-04-22,Ruud C.,Ruusuvuori E.,7.0,73.0,4110.0,856.0,1.16,5.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15462,0,2022-04-22,Alcaraz C.,Tsitsipas S.,11.0,5.0,3330.0,5980.0,1.66,2.20,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
15463,0,2022-04-23,Rublev A.,Fognini F.,8.0,62.0,3865.0,906.0,1.33,3.40,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
15464,1,2022-04-23,Djokovic N.,Khachanov K.,26.0,1.0,8340.0,1566.0,4.00,1.25,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


### [NEW] Save all dataframe into CSV 

In [245]:
df_serve_leaderboard.to_csv(r'/path/serve_leaderboard_scrapped.csv',
                  index=False, decimal=".")
df_return_leaderboard.to_csv(r'/path/return_leaderboard_scrapped.csv',
                  index=False, decimal=".")
df_under_pressure_leaderboard.to_csv(r'/path/under_pressure_leaderboard_scrapped.csv',
                  index=False, decimal=".")
df_rankings.to_csv(r'/path/rankings_scrapped.csv',
                  index=False, decimal=".")